In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))'''

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

"for dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))"

In [2]:
from tensorflow.keras.applications.vgg16 import VGG16

In [3]:
model_vgg=VGG16(include_top=False,input_shape=(256,256,3))

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [5]:
for layer in model_vgg.layers:
    layer.trainable=False

In [6]:
model_vgg.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [7]:
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model

In [8]:
last_layer=Dense(2,activation='softmax')(Flatten()(model_vgg.output))

In [9]:
model=Model(inputs=model_vgg.input,outputs=last_layer)

model.compile(loss='categorical_crossentropy',metrics=['accuracy'])

# Data preparation

In [10]:
filenames=os.listdir('/kaggle/input/dogs-vs-cats/train/train')

In [11]:
len(filenames)

25000

In [12]:
filenames[0:5]

['cat.12461.jpg',
 'dog.3443.jpg',
 'dog.7971.jpg',
 'dog.10728.jpg',
 'dog.1942.jpg']

In [13]:
classes=[file[:3] for file in filenames]
classes[:5]

['cat', 'dog', 'dog', 'dog', 'dog']

In [14]:
df=pd.DataFrame({'filename':filenames,'class':classes})
df.head()

,filename,class
0,cat.12461.jpg,cat
1,dog.3443.jpg,dog
2,dog.7971.jpg,dog
3,dog.10728.jpg,dog
4,dog.1942.jpg,dog


In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
data_gen=ImageDataGenerator(zoom_range=0.20,shear_range=0.20,rescale=1/255,horizontal_flip=True)

In [17]:
train_data=data_gen.flow_from_dataframe(df,'/kaggle/input/dogs-vs-cats/train/train/',
                             x_col='filename',y_col='class',class_mode='categorical')

Found 25000 validated image filenames belonging to 2 classes.


In [18]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [19]:
train_data.class_indices

{'cat': 0, 'dog': 1}

In [20]:
model.fit(train_data,epochs=5)

Epoch 1/5
782/782 [==============================] - 570s 716ms/step - loss: 0.6511 - accuracy: 0.8400
Epoch 2/5
782/782 [==============================] - 405s 518ms/step - loss: 0.4545 - accuracy: 0.8952
Epoch 3/5
782/782 [==============================] - 406s 519ms/step - loss: 0.3986 - accuracy: 0.9079
Epoch 4/5
782/782 [==============================] - 407s 521ms/step - loss: 0.3854 - accuracy: 0.9165
Epoch 5/5
782/782 [==============================] - 407s 520ms/step - loss: 0.3697 - accuracy: 0.9210


In [21]:
model.save('/kaggle/working/cat_dog_model.h5')